In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import cv2
import tensorflow as tf
import seaborn as sn
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import Model
from keras.utils import plot_model
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split


print(tf.__version__)

In [ ]:
training = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')

In [ ]:
# training = training.sort_values(by=["target"])
# training.head()

Get equal split of benign and malignant

In [ ]:
# m = training[32542:]
# b = training[:32542].sample(2000)
# print(m.shape)
# print(b.shape)

In [ ]:
# min = m["age_approx"].min()
# max = m["age_approx"].max()
# diff = max-min

In [ ]:
# img_size = (224,224)
# m_img = []
# m_aux = []
# for file, sex, age in tqdm(zip(m["image_name"], m["sex"], m["age_approx"])):
#     img = plt.imread(f"../input/siim-isic-melanoma-classification/jpeg/train/{file}.jpg")
#     img = cv2.resize(img, img_size)
#     m_img.append(img)
#     aux = [0,0,0]
#     if (sex == None or age == None):
#         break
#     if (sex == "male"):
#         aux[0] = 1
#     else:
#         aux[1] = 1
#     aux[2] = (age-min)/diff
#     m_aux.append(aux)
    
# m_img = np.array(m_img)
# m_aux = np.array(m_aux)
# print(m_img.shape)
# print(m_aux.shape)

In [ ]:
# b_img = []
# b_aux = []
# for file, sex, age in tqdm(zip(b["image_name"], b["sex"], b["age_approx"])):
#     img = plt.imread(f"../input/siim-isic-melanoma-classification/jpeg/train/{file}.jpg")
#     img = cv2.resize(img, img_size)
#     b_img.append(img)
#     aux = [0,0,0]
#     if (sex == None or age == None):
#         break
#     if (sex == "male"):
#         aux[0] = 1
#     else:
#         aux[1] = 1
#     aux[2] = (age-min)/diff
#     b_aux.append(aux)
    
# b_img = np.array(b_img)
# b_aux = np.array(b_aux)
# print(b_img.shape)
# print(b_aux.shape)

In [ ]:
# with open("m_img.npz", "wb") as file:
#     np.savez_compressed(file, images=m_img)
    
# with open("b_img.npz", "wb") as file:
#     np.savez_compressed(file, images=b_img)  


In [ ]:
# with open("m_aux.npz", "wb") as file:
#     np.savez_compressed(file, aux=m_aux)
    
# with open("b_aux.npz", "wb") as file:
#     np.savez_compressed(file, aux=b_aux)  


In [ ]:
with open("../input/previousvgg/m.npz", "rb") as file:
    m_img = np.load(file)["images"]
with open("../input/previousvgg/b.npz", "rb") as file:
    b_img = np.load(file)["images"]
with open("../input/previousvgg/m_aux.npz", "rb") as file:
    m_aux = np.load(file)["aux"]
with open("../input/previousvgg/b_aux.npz", "rb") as file:
    b_aux = np.load(file)["aux"]

In [ ]:
imgs = np.concatenate((b_img, m_img))
auxs = np.concatenate((b_aux, m_aux))
labels = np.concatenate((np.full(2000,0), np.full(584,1)))

del b_img, m_img, b_aux, m_aux
gc.collect()

indices = np.arange(labels.shape[0])
np.random.seed(0)
np.random.shuffle(indices)

imgs = imgs[indices]
auxs = auxs[indices]
labels = labels[indices]


print(imgs.shape)
print(auxs.shape)
print(labels.shape)

In [ ]:
trainX, valX, aux_trainX, aux_valX, trainY, valY = train_test_split(
    imgs, auxs, 
    labels,
    test_size = 0.2, 
    random_state = 0
)

print(trainX.shape)
print(aux_trainX.shape)
print(trainY.shape)
print(valX.shape)
print(aux_valX.shape)
print(valY.shape)

del imgs
del labels
gc.collect()

In [ ]:
# aug = ImageDataGenerator(
#         rescale=1./255,
#         rotation_range=90,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         zoom_range=(0.8,1.2),
#         horizontal_flip=True,
#         vertical_flip = True,
#         brightness_range=(0.8,1.2))

train_aug_args = dict(rescale=1./255,
                     rotation_range=20,
                     width_shift_range=0.2, 
                     height_shift_range=0.2,
                     horizontal_flip=True,
                     vertical_flip=True)
val_aug_args = dict(rescale=1./255)

aug = ImageDataGenerator()


In [ ]:
def custom_gen(images, auxs, labels, batch_size, val): #batch size must be less than length of all data    
    i=0
    np.random.seed(0)
    while True:
        batch = {'images': [], 'auxs': [], 'labels': []}
        for b in range(batch_size):
            if i == len(images):
                i = 0
                indices = np.arange(labels.shape[0]) #generate 0..# of images                
                np.random.shuffle(indices)
                images = images[indices] #shuffle arrays
                auxs = auxs[indices]
                labels = labels[indices]
            if (val):
                batch["images"].append(aug.apply_transform(images[i], val_aug_args))
            else:
                batch["images"].append(aug.apply_transform(images[i], train_aug_args))
            batch["auxs"].append(auxs[i])
            batch["labels"].append(labels[i])
            i+=1

        batch['images'] = np.array(batch['images'])
        batch['auxs'] = np.array(batch['auxs'])
        batch['labels'] = np.array(batch['labels'])

        yield [batch['images'], batch['auxs']], batch['labels']

In [ ]:
# train_gen = train_aug.flow(trainX, trainY, batch_size = 1, shuffle = True)
# val_gen = val_aug.flow(valX, valY, batch_size = 8, shuffle = False)


In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint
# early_stop = EarlyStopping(monitor='val_loss', patience=4)
# checkpoint = ModelCheckpoint("{val_loss:.2f}-{epoch:02d}.hdf5",monitor = 'val_loss',verbose = 1,save_best_only = True,mode = 'min')

# callbacks = [early_stop, checkpoint]

In [ ]:
from tensorflow.python.keras import backend as K
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
optimizer = keras.optimizers.Adam(lr=1e-5)

In [ ]:
vgg = VGG19(include_top=False, weights='imagenet', input_shape = (224,224,3))
flat = Flatten()(vgg.layers[-1].output)

aux_in = Input(shape=(3,), name="aux_in")
aux = Dense(3, activation = "relu", name="aux1")(aux_in)

merged = concatenate([flat, aux])
merged = Dense(1, activation="sigmoid")(merged)


In [ ]:
model = Model(inputs=[vgg.input, aux_in], outputs=[merged])
# plot_model(model,to_file='demo.png',show_shapes=True)

In [ ]:
# model = keras.Sequential()
# model.add(VGG19(include_top=False, weights='imagenet', input_shape = (224,224,3)))
# model.add(layers.Flatten())
# model.add(layers.Dense(1, activation="sigmoid"))

# model.summary()

In [ ]:
# model.add(layers.Dense(32))
# model.add(layers.LeakyReLU(0.001))
# model.add(layers.Dense(16))
# model.add(layers.LeakyReLU(0.001))
# model.add(layers.Dense(1, activation='sigmoid'))
# model.layers[0].trainable = False

model.compile(loss=focal_loss(), metrics=[tf.keras.metrics.AUC()], optimizer=optimizer)

history = model.fit_generator(custom_gen(
    images = trainX,
    auxs = aux_trainX,
    labels = trainY,
    batch_size=8,
    val=False),
    steps_per_epoch = trainX.shape[0] // 8,
    epochs = 2, 
    validation_data = custom_gen(
    images = valX,
    auxs = aux_valX,
    labels = valY,
    batch_size=8,
    val=True),
    validation_steps = valX.shape[0] // 8
#     callbacks = callbacks
)

model.save('melanoma-classification-model-aux.h5')

In [ ]:
del trainX, trainY, aux_trainX, aux_valX, valX, valY
gc.collect()

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(hist['epoch'], hist['loss'], label='Train Error')
plt.plot(hist['epoch'], hist['val_loss'], label='Val Error')
plt.ylim([0, 1])
plt.legend()

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.plot(hist['epoch'], hist['auc_2'], label='Train Acc')
plt.plot(hist['epoch'], hist['val_auc_2'], label='Val Acc')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
testing = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
testing.head()

In [ ]:
test_images = np.load('../input/siimisic-melanoma-resized-images/x_test_224.npy')
predictions = []
t_auxs = []

for img, sex, age in tqdm(zip(test_images, testing["sex"], testing["age_approx"])):    
    t_imgs.append(img)
    img=np.reshape(img,(1,224,224,3))
    aux = [0,0,0]
    if (sex == "male"):
        aux[0] = 1
    else:
        aux[1] = 1
    aux[2] = (age-15)/75 #max age is 90, min age is 15
    aux = np.reshape(np.array(aux), (1,3))        
    t_auxs.append(aux)
    pred=model.predict([img, aux])
    predictions.append(pred[0][0])
    
t_auxs = np.array(t_auxs)
print(t_auxs.shape)

In [ ]:
print(t_auxs[:10])

In [ ]:
with open("t_aux.npz", "wb") as file:
    np.savez_compressed(file, aux=t_auxs)


In [ ]:
del hist

In [ ]:
print(predictions[:10])

In [ ]:
results = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
results['target'] = predictions
results.to_csv('vgg_predictions5.csv', header=True, index=False)

In [ ]:
results.head()

In [ ]:
temp = pd.read_csv('../input/previousvgg/vgg_predictions(1).csv')
temp.head()

In [ ]:
res = np.absolute(temp["target"]-results["target"])
print(res.shape)
print(np.sum(res))

In [ ]:
# vggmodel = VGG19(include_top=False, weights="imagenet")
# print(vggmodel.summary())

In [ ]:
# def get_model():
#     model = keras.Sequential([
#         layers.Flatten(input_shape=(7, 7, 512)),
#         layers.Dense(64, activation=tf.nn.relu),
#         layers.Dropout(0.25),
#         layers.Dense(1, activation=tf.nn.sigmoid)
#     ])

#     model.compile(optimizer='adam',
#                   loss='binary_crossentropy',
#                   metrics=['accuracy', 'binary_crossentropy'])
#     return model

In [ ]:
# with open("../input/vggconv/VGGCompressedData4.npz", "rb") as file:
#     aug_convs = np.load(file)["images"]

# print(aug_convs.shape)

In [ ]:
# with open("../input/vggconv/VGGCompressedData1.npz", "rb") as file:
#     convs = np.load(file)["images"]
# labels = training["target"].values[:15000]

# print(convs.shape)
# print(labels.shape)

In [ ]:
# convs = np.concatenate((convs, aug_convs[:3500]))
# labels = np.concatenate((labels, np.full(3500,1)))

# print(convs.shape)
# print(labels.shape)

In [ ]:
# epoch = 2
# top = get_model()
# history = top.fit(convs,labels,epochs=epoch, validation_split=0.1,verbose=2)


In [ ]:
# hist = pd.DataFrame(history.history)
# hist['epoch'] = history.epoch

# plt.figure()
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.plot(hist['epoch'], hist['loss'], label='Train Error')
# plt.plot(hist['epoch'], hist['val_loss'], label='Val Error')
# plt.ylim([0, 1])
# plt.legend()

# plt.figure()
# plt.xlabel('Epoch')
# plt.ylabel('accuracy')
# plt.plot(hist['epoch'], hist['accuracy'], label='Train Acc')
# plt.plot(hist['epoch'], hist['val_accuracy'], label='Val Acc')
# plt.ylim([0, 1])
# plt.legend()
# plt.show()

In [ ]:
# del convs
# del labels
# gc.collect()

In [ ]:
# with open("../input/vggconv/VGGCompressedData2.npz", "rb") as file:
#     convs = np.load(file)["images"]
# labels = training["target"].values[15000:]

# print(convs.shape)
# print(labels.shape)

In [ ]:
# convs = np.concatenate((convs, aug_convs[3500:]))
# labels = np.concatenate((labels, np.full(3808,1)))

# print(convs.shape)
# print(labels.shape)


In [ ]:
# del aug_convs
# gc.collect()

In [ ]:
# history = top.fit(convs,labels,epochs=epoch, validation_split=0.1,verbose=2)

In [ ]:
# hist = pd.DataFrame(history.history)
# hist['epoch'] = history.epoch

# plt.figure()
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.plot(hist['epoch'], hist['loss'], label='Train Error')
# plt.plot(hist['epoch'], hist['val_loss'], label='Val Error')
# plt.ylim([0, 0.2])
# plt.legend()

# plt.figure()
# plt.xlabel('Epoch')
# plt.ylabel('accuracy')
# plt.plot(hist['epoch'], hist['accuracy'], label='Train Acc')
# plt.plot(hist['epoch'], hist['val_accuracy'], label='Val Acc')
# plt.ylim([0, 1])
# plt.legend()
# plt.show()

In [ ]:
# del convs
# del labels
# gc.collect()

In [ ]:
# top.save('../working/melanoma_vgg19_2.h5')

In [ ]:
# with open("../input/vggconv/VGGCompressedData3.npz", "rb") as file:
#     convs = np.load(file)["images"]
# print(convs.shape)

In [ ]:
# prediction = top.predict(convs)
# print(prediction.shape)

In [ ]:
# results = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
# results['target'] = prediction[:,0]
# results.to_csv('vgg_predictions.csv', header=True, index=False)

In [ ]:
# results.head(10)